Learning All laws on a 1D grid
============================
Using Wolfram's 256 rules of Cellular Automata

In [1]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import tensorflow as tf

2022-01-25 12:13:32.882714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-25 12:13:32.882815: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Learning Simple Laws

## Constant motion  
(replaced by Wolfram's rules since it is one of them)

In [259]:
L=10 #length of 1-d box

def cm_update(x):
    L=len(x)
    y=x.copy()
    for i in range(L):
        y[i]=x[(i-1)%L]
    return(y)


## Wolfram's 256 Rules

These are all cellular automaton rules for a 1-d array with only neares neighbour interaction. There are therefore only three cells (L,C,R) determining the outcome of one cell (C). This gives 2^3=8 possible combinations that can give rise to an outcome of 2 possibilities. Thus there are 2^8=256 possible rules. 

The following function generates each of these rules.
 
Input:  
num = number of rule  
inp = string of input bits   
t = number of timesteps calculated  

Output: an array of arrays each for one timestep beginning with the input array. 

The boudary conditions are periodic
        

In [220]:
def CellularAutomata(num,inp,t):
    bnum = np.flip(np.fromiter(np.binary_repr(num,width=8),dtype=int))
    out = [np.fromiter(inp,dtype=int)]
    lin = len(inp)
    currinp = inp
    for n in range(t):
        outnow = []
        for i in range(len(inp)):
            outnow = np.append(outnow,bnum[int(str(currinp[(i-1)%lin])+str(currinp[(i)%lin])+str(currinp[(i+1)%lin]),2)])

        currinp =''.join([str(int(elem)) for elem in outnow])
        out = np.append(out,[outnow],axis=0)
    return(out)


## Learning

In [461]:
N=10000 #batch size
L=3 #Length of box
rule= 1
time = 1

x_train=[]
y_train=[]


for i in range(N):
    x=np.random.randint(0,2,L)
    x_train.append(x)
    y_train.append(CellularAutomata(rule,''.join([str(int(elem)) for elem in x]),time)[time,2])

    
x_train = np.array(x_train).reshape(N,L)
y_train = np.array(y_train).reshape(N,1)

Ok, using the above data we don't get any reasonable results. For some reason the machine only reproduces statistics of the training set. Eg. for rule 1 it the achieved accuracy is exaclty the occurence of the input "000", which is the only one that does not result in 0 for the middle cell.

In [465]:
#Counting the number of occurence of certain elements
count=0
for x in x_train:
  #  if all(x == [0,0,1]) or all(x == [0,0,0]) or all(x == [1,0,0]) or all(x == [1,1,0]) or all(x == [1,1,1]):
    if all(x == [0,0,0]):
         
        count+=1
print(count/N)

0.1237


In [463]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1),
    tf.keras.layers.Dense(6),
    tf.keras.layers.Dense(6),
    tf.keras.layers.Dense(1,activation="softmax")
])

In [464]:
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
predictions = model(x_train).numpy()

loss_fn(y_train, predictions).numpy()

model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=[tf.keras.metrics.BinaryAccuracy()])
model.fit(x_train, y_train, epochs=6)
pred=model(x_train).numpy()

Epoch 1/6
313/313 [==============================] - 1s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237
Epoch 2/6
313/313 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237
Epoch 3/6
313/313 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237
Epoch 4/6
313/313 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237
Epoch 5/6
313/313 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237
Epoch 6/6
313/313 [==============================] - 0s 1ms/step - loss: 0.0000e+00 - binary_accuracy: 0.1237


## Exact predictions

In [389]:
data=100
print("x_train: ",x_train[data])
print("y_train: ",y_train[data])
a=np.array(x_train[data]).reshape(1,L)
print("Machine: ",np.rint(model(a).numpy())) #rounding to integer

x_train:  [0 0 1]
y_train:  [1.]
Machine:  [[1.]]


In [404]:
inp="001"
print("in: ",inp)
print("out: ", CellularAutomata(110,inp,1)[1])
a=np.array([0, 0, 1]).reshape(1,L)
print("Machine: ",np.rint(model(a).numpy())) #rounding to integer

in:  001
out:  [0. 1. 1.]
Machine:  [[1.]]


In [397]:
CellularAutomata(110,"000",1)

array([[0., 0., 0.],
       [0., 0., 0.]])

This does not seem to work